In [11]:
import polars as pl
from pathlib import Path

In [12]:
benchmark_path = '/s/project/rep/processed/training_results_v15/gtex_benchmark_with_annotation.parquet/*.parquet'
genotypes_path = '/s/project/rep/processed/training_results_v15/gtex_v8_old_dna/private_variants.parquet/rare_variants.vcf.parquet/**/*.parquet'
tss_variant_path = '/data/nasif12/home_if12/tsi/kipoi_expression_prediction/variant_tables/tss_variants_50_500.parquet'
apa_variant_path = '/data/nasif12/home_if12/tsi/kipoi_expression_prediction/variant_tables/apa_variants_70_130.parquet'

In [13]:
tss_variant_df = pl.scan_parquet(tss_variant_path).with_columns(
    pl.col('gene_id').str.replace(r'([^\.]+)\..+$', "${1}").alias('gene'),
).select(['gene', 'chrom', 'variant_start', 'variant_end', 'ref', 'alt'])
apa_variant_df = pl.scan_parquet(apa_variant_path).with_columns(
    pl.col('gene_id').str.replace(r'([^\.]+)\..+$', "${1}").alias('gene'),
).select(['gene', 'chrom', 'variant_start', 'variant_end', 'ref', 'alt'])
benchmark_df = pl.scan_parquet(benchmark_path)
genotypes_df = pl.scan_parquet(genotypes_path, hive_partitioning=True).select(
    ['sampleId', 'chrom', 'start', 'end', 'ref', 'alt']).rename(
    {'sampleId': 'individual',
     'start': 'variant_start',
     'end': 'variant_end', })

In [14]:
tss_variant_df.head().collect()

gene,chrom,variant_start,variant_end,ref,alt
str,str,i64,i64,str,str
"""ENSG00000184481""","""chrX""",70315840,70315841,"""C""","""G"""
"""ENSG00000184481""","""chrX""",70315994,70315995,"""A""","""G"""
"""ENSG00000184481""","""chrX""",70316097,70316098,"""T""","""C"""
"""ENSG00000184481""","""chrX""",70316097,70316098,"""T""","""C"""
"""ENSG00000184481""","""chrX""",70319566,70319567,"""A""","""T"""


In [15]:
apa_variant_df.head().collect()

gene,chrom,variant_start,variant_end,ref,alt
str,str,i64,i64,str,str
"""ENSG00000184481""","""chrX""",70320652,70320653,"""C""","""T"""
"""ENSG00000184481""","""chrX""",70320652,70320653,"""C""","""T"""
"""ENSG00000184634""","""chrX""",70344989,70344990,"""C""","""T"""
"""ENSG00000184634""","""chrX""",70346868,70346869,"""G""","""A"""
"""ENSG00000184634""","""chrX""",70346868,70346869,"""G""","""A"""


In [16]:
benchmark_df.head().collect()

individual,gene,tissue,tissue_type,mu,theta,counts,normalized_counts,l2fc,zscore,nominal_pvalue,FDR,is_in_benchmark,is_underexpressed_outlier,AbExp,expected_expr@theta,splice_ensemble@AbSplice,splice_ensemble@MMSplice_SpliceMap_Psi_ref,splice_ensemble@SpliceAI,vep@cadd_raw.max,vep@LoF_HC.max,vep@transcript_ablation.max,vep@stop_gained.max,vep@frameshift_variant.max,vep@coding_sequence_variant.max,vep@missense_variant.max,vep@inframe_deletion.max,vep@inframe_insertion.max,vep@stop_lost.max,vep@3_prime_UTR_variant.max,vep@5_prime_UTR_variant.max,vep@NMD_transcript_variant.max,vep@NMD_escaping_variant.max,vep@start_lost.max,vep@splice_donor_variant.max,vep@splice_acceptor_variant.max,vep@splice_region_variant.max,is_obvious_outlier
str,str,str,str,f32,f32,i32,f64,f32,f32,f64,f64,bool,bool,f64,f32,f64,f64,f64,f32,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
"""GTEX-1117F""","""ENSG00000000003""","""Adipose - Subcutaneous""","""Adipose Tissue""",1135.241089,78.591827,1173,1653.796398,0.047163,0.323383,0.752009,1.0,true,false,-0.002567,78.591827,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,false
"""GTEX-1117F""","""ENSG00000000003""","""Adipose - Visceral (Omentum)""","""Adipose Tissue""",1673.81189,45.643147,1874,1684.855628,0.162892,0.818604,0.415125,1.0,true,false,-0.002567,45.643147,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,false
"""GTEX-1117F""","""ENSG00000000003""","""Artery - Coronary""","""Blood Vessel""",958.796692,54.65052,1414,718.418106,0.560001,3.042509,0.002392,1.0,true,false,-0.002567,54.65052,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,false
"""GTEX-1117F""","""ENSG00000000003""","""Breast - Mammary Tissue""","""Breast""",2295.427734,59.121643,2476,1993.721864,0.109203,0.627099,0.532658,1.0,true,false,-0.002567,59.121643,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,false
"""GTEX-1117F""","""ENSG00000000003""","""Minor Salivary Gland""","""Salivary Gland""",1206.162109,83.073303,1290,2047.137887,0.09687,0.639452,0.52709,1.0,true,false,-0.002567,83.073303,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,false


In [17]:
genotypes_df.head().collect()

individual,chrom,variant_start,variant_end,ref,alt
str,str,i64,i64,str,str
"""GTEX-1117F""","""chr1""",133957,133960,"""CCT""","""C"""
"""GTEX-1117F""","""chr1""",234426,234434,"""GACACACC""","""G"""
"""GTEX-1117F""","""chr1""",234436,234437,"""A""","""ATG"""
"""GTEX-1117F""","""chr1""",532044,532045,"""C""","""A"""
"""GTEX-1117F""","""chr1""",819127,819128,"""G""","""A"""


In [18]:
tss_genotypes_df = genotypes_df.join(tss_variant_df, on=['chrom', 'variant_start', 'variant_end', 'ref', 'alt'], how='inner')
tss_genotypes_df.head().collect()

individual,chrom,variant_start,variant_end,ref,alt,gene
str,str,i64,i64,str,str,str
"""GTEX-1117F""","""chr1""",3566597,3566598,"""G""","""A""","""ENSG00000116213"""
"""GTEX-1117F""","""chr1""",3566597,3566598,"""G""","""A""","""ENSG00000116213"""
"""GTEX-1117F""","""chr1""",3566597,3566598,"""G""","""A""","""ENSG00000116213"""
"""GTEX-1117F""","""chr1""",3566597,3566598,"""G""","""A""","""ENSG00000116213"""
"""GTEX-1117F""","""chr1""",3566597,3566598,"""G""","""A""","""ENSG00000116213"""


In [19]:
apa_genotypes_df = genotypes_df.join(apa_variant_df, on=['chrom', 'variant_start', 'variant_end', 'ref', 'alt'], how='inner')
apa_genotypes_df.head().collect()

individual,chrom,variant_start,variant_end,ref,alt,gene
str,str,i64,i64,str,str,str
"""GTEX-1117F""","""chr1""",8073460,8073461,"""G""","""A""","""ENSG00000116285"""
"""GTEX-1117F""","""chr1""",15687051,15687052,"""G""","""A""","""ENSG00000142621"""
"""GTEX-1117F""","""chr1""",15773112,15773113,"""C""","""T""","""ENSG00000162438"""
"""GTEX-1117F""","""chr1""",20412776,20412777,"""T""","""C""","""ENSG00000127472"""
"""GTEX-1117F""","""chr1""",25170827,25170828,"""A""","""AT""","""ENSG00000169504"""


In [20]:
apa_gene_sample_df = apa_genotypes_df.select(['individual', 'gene']).unique()
tss_gene_sample_df = tss_genotypes_df.select(['individual', 'gene']).unique()

In [21]:
tss_benchmark_df = benchmark_df.join(tss_gene_sample_df, on=['individual', 'gene'], how='inner').collect()

In [ ]:
apa_benchmark_df = benchmark_df.join(apa_gene_sample_df, on=['individual', 'gene'], how='inner').collect()

In [ ]:
benchmark_df.select(pl.len()).collect()

In [ ]:
apa_benchmark_df.shape

In [ ]:
tss_benchmark_df.shape

In [ ]:
apa_benchmark_df.select(pl.col('is_underexpressed_outlier').sum())

In [ ]:
tss_benchmark_df.select(pl.col('is_underexpressed_outlier').sum())

In [23]:
output_dir = Path('/data/nasif12/home_if12/tsi/kipoi_expression_prediction/new_benchmarks')
output_dir.mkdir(exist_ok=True)

In [24]:
tss_benchmark_df.write_parquet(output_dir / 'tss_benchmark.parquet')

In [ ]:
apa_benchmark_df.write_parquet(output_dir / 'pas_benchmark.parquet')